<img src='https://i.imgur.com/QmqZv8W.png' width=150%>
<div class='info'>
Epidemiological models of COVID-19 forecast the number of future cases and deaths.
Quantifying the accuracy of these models can help to inform public health decisions.
    <b>COVIDForeca.st</b> scores the predictive ability of publicly available COVID-19 epidemiological models on the <a href=https://covid19forecasthub.org>COVID-19 Forecast Hub</a>.
Our scoring system uses the posted forecast cumulative distributions to compute the log-likelihood for held-out COVID-19 positive cases and deaths.
Scores are updated continuously as new data become available, and model performance is tracked over time.
The leaderboard considers predictions from July 4th, 2020 and beyond, and includes only those models covering at least half of that timespan.
</div>

In [46]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
import ipywidgets as ipyw
from ipywidgets import HBox, VBox, Text, HTML, Label, AppLayout, IntSlider, IntRangeSlider, Output
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import qgrid
import seaborn as sns
import sys
sns.set(font_scale=1)
sns.set_style('whitegrid')

sys.path.insert(0, '..')
import Scoreboard19 as S19

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
#US_cases = S19.read_observed('cases', writetocsv=True)
#US_deaths = S19.read_observed('deaths', writetocsv=True)
US_cases = S19.read_observed('cases', use_cache=True)
US_deaths = S19.read_observed('deaths', use_cache=True)
scoreboard_cases = pd.read_pickle(S19.data_dir / 'ScoreboardCases.pkl')
scoreboard_deaths = pd.read_pickle(S19.data_dir / 'ScoreboardDeaths.pkl')
scoreboard_cases = S19.fix_scoreboard(scoreboard_cases, kind='Case', quiet=True, plot=False)
scoreboard_deaths = S19.fix_scoreboard(scoreboard_deaths, kind='Death', quiet=True, plot=False)

In [62]:
def get_qgrid(scoreboard, weeks_ahead):
    df = S19.getleaderboard(scoreboard, weeks_ahead, quiet=True)
    df.columns = [x.title().replace('Of ', '').replace('Rankings', 'Rank').replace('Average', 'Mean').replace('Past ', '').replace('Scores', 'Score')
                  for x in df.columns]
    df.iloc[:, 1] = df.iloc[:, 1].round(2)
    df.iloc[:, 2] = df.iloc[:, 2].round(1)
    df['Team'] = df['Model'].apply(lambda x: x.split(':')[0])
    df['Model'] = df['Model'].apply(lambda x: x.split(':')[1])
    df = df.set_index('Team')
    df = df.drop(['Deltaw', 'Forecasttype', 'Asofdate'], axis=1)
    #df.index.name = "Team:Model"
    return qgrid.show_grid(df, grid_options={'width': 450, 'forceFitColumns': True, 'defaultColumnWidth': 110})

output_q_cases = Output()
output_q_deaths = Output()

def slide_1(slider_value):
    weeks_ahead = slider_value['new']
    
    with output_q_cases:
        q_cases = get_qgrid(scoreboard_cases, weeks_ahead)
        q_cases.layout = ipyw.Layout(padding='10px', width='458px', align_items='center')
        clear_output(wait=True)
        display(q_cases)
        
slider_1 = IntSlider(value=3, min=1, max=6, description='Leaderboard Weeks Ahead', continuous_update=False)
slider_1.style = {'description_width': 'initial'}
slider_1.layout = ipyw.Layout(align_items='center', align_self='center', height='100px', width='90%')
slider_1.observe(slide_1, names='value')
centered = ipyw.Layout(align_items='center', align_self='center',
                       padding='1px', border='1px', margin='1px')
title_1 = Label('COVIDHub Model Rankings')
d_1 = VBox([slider_1, output_q_cases, Label('Cases')], layout=centered)
display(d_1)
slider_1.value = 3
slider_1.value = 4

display(HTML("""
<style>
.widget-label {
    text-align: center;
    font-size: 150%;
}
.widget-readout {
    text-align: center;
    font-size: 150%;
}
.q-grid-container {
    text-align: center;
}
.info {
    font-size: 125%;
}
.caption {
    font-size: 125%;
}
</style>
"""))

In [63]:
output_q_deaths = Output()

def slide_2(slider_value):
    weeks_ahead = slider_value['new']
    
    with output_q_deaths:
        q_deaths = get_qgrid(scoreboard_deaths, weeks_ahead)
        q_deaths.layout = ipyw.Layout(padding='10px', width='458px', align_items='center')
        clear_output(wait=True)
        display(q_deaths)
        
slider_2 = IntSlider(value=3, min=1, max=6, description='Leaderboard Weeks Ahead', continuous_update=False)
slider_2.style = {'description_width': 'initial'}
slider_2.layout = ipyw.Layout(align_items='center', align_self='center', height='100px', width='90%')
slider_2.observe(slide_2, names='value')
centered = ipyw.Layout(align_items='center', align_self='center', padding='1px', border='1px', margin='1px')
d_2 = VBox([slider_2, output_q_deaths, Label('Deaths')], layout=centered)
display(d_2)
slider_2.value = 3
slider_2.value = 4

In [64]:
title_3 = Label('Forecasts and Scores')
slider_3 = IntSlider(value=2, min=1, max=6, description='Weeks Ahead', continuous_update=False)
slider_3.style = {'description_width': 'initial'}
slider_3.layout = ipyw.Layout(align_items='center', align_self='center', height='100px', width='90%')
out_3 = Output()

def slide_3(x):
    weeks_ahead = x['new']
    with out_3:
        clear_output(wait=True)
        S19.plotTD(scoreboard_cases, weeks_ahead, ['FDANIH:Sunweight', 'FDANIH:Sweight'])
        plt.show()
    
slider_3.observe(slide_3, names='value')
d_3 = VBox([slider_3, out_3], layout=centered)
display(d_3)
slider_3.value = 4

In [65]:
title_4 = Label('Forecasts and Scores')
slider_4 = IntSlider(value=2, min=1, max=6, description='Weeks Ahead', continuous_update=False)
slider_4.style = {'description_width': 'initial'}
slider_4.layout = ipyw.Layout(align_items='center', align_self='center', height='100px', width='90%')
out_4 = Output()

def slide_4(x):
    weeks_ahead = x['new']
    with out_4:
        clear_output(wait=True)
        S19.plotlongitudinal(US_cases, scoreboard_cases, 'Cases', weeks_ahead, ['COVIDhub:ensemble', 'FDANIH:Sweight'])
        plt.show()
    
slider_4.observe(slide_4, names='value')
d_4 = VBox([slider_4, out_4], layout=centered)
display(d_4)
slider_4.value = 4

In [41]:
title_5 = Label('Forecast Estimates')
max_weeks_ahead = 7
slider_5 = IntRangeSlider(value=[2, 6], min=1, max=max_weeks_ahead, description='Weeks Ahead', continuous_update=False)
slider_5.style = {'description_width': 'initial'}
slider_5.layout = ipyw.Layout(align_items='center', align_self='center', height='100px', width='70%')
out_5 = Output()

def slide_5(x):
    numweeks_start, numweeks = x['new']
    with out_5:
        clear_output(wait=True)
        S19.plotlongitudinalUNWEIGHTED(US_cases, scoreboard_cases, 'Cases', numweeks,
                                       numweeks_start=numweeks_start, max_weeks_ahead=max_weeks_ahead+1)
        plt.show()
    
slider_5.observe(slide_5, names='value')
d_5 = VBox([slider_3, out_5], layout=centered)
display(d_5)
slider_5.value = (1, max_weeks_ahead)
html = """<div class='caption'>
<b>Forecast performance over time.</b>
TOP FIGURES: Black solid curves represent the observed US weekly incidental case counts.
Other curves represent the median of the forecasts for the target end date made from
1 to 7-weeks prior to the target end dates.
BOTTOM FIGURES: Curves represent the median of the forecast scores colored based on their
forecasting horizon (1-week prior to 7-weeks prior color-matching to figures above). 
NOTE: Discontinuities in the score plots imply that the median value of the scores
for a particular time-horizon is $-\infty$, demonstrating the poor performance especially
on the inflection points of the epidemiological curves.
</div>"""
display(HTML(html))

In [42]:
title_6 = Label('Forecast Estimates')
max_weeks_ahead = 7
slider_6 = IntRangeSlider(value=[2, 6], min=1, max=max_weeks_ahead, description='Weeks Ahead', continuous_update=False)
slider_6.style = {'description_width': 'initial'}
slider_6.layout = ipyw.Layout(align_items='center', align_self='center', height='100px', width='70%')
out_6 = Output()

def slide_6(x):
    numweeks_start, numweeks = x['new']
    with out_6:
        clear_output(wait=True)
        S19.plotlongitudinalUNWEIGHTED(US_deaths, scoreboard_deaths, 'Deaths', numweeks,
                                       numweeks_start=numweeks_start, max_weeks_ahead=max_weeks_ahead+1)
        plt.show()
    
slider_6.observe(slide_6, names='value')
d_6 = VBox([slider_6, out_6], layout=centered)
display(d_6)
slider_6.value = (1, max_weeks_ahead)
html = """<div class='caption'>
<b>Forecast performance over time.</b>
TOP FIGURES: Black solid curves represent the observed US weekly incidental death counts.
Other curves represent the median of the forecasts for the target end date made
from 1 to 7-weeks prior to the target end dates.
BOTTOM FIGURES: Curves represent the median of the forecast scores colored based on their
forecasting horizon (1-week prior to 7-weeks prior color-matching to figures above). 
NOTE: Discontinuities in the score plots imply that the median value of the scores
for a particular time-horizon is $-\infty$, demonstrating the poor performance especially
on the inflection points of the epidemiological curves.
</div>"""
display(HTML(html))

In [71]:
out_7 = Output(layout=centered)
with out_7:
    clear_output(wait=True)
    S19.plotallscoresdist(scoreboard_deaths, 'Case', interval='Weeks')
    plt.show()
d_7 = VBox([out_7], layout=centered)
display(d_7)
html = """
<div class='caption'>
<b>Scores over Time (Cases)</b>.
(TOP) Scatter plot for all scores as a function of the forecast horizon
(Weekly incidental case forecast scores).
(BOTTOM) Histogram of weekly incidental case count forecasts.
</div>
"""
HTML(html)

In [45]:
#title_4 = Label('Scores over Time')
out_8 = Output(layout=centered)
with out_8:
    clear_output(wait=True)
    S19.plotallscoresdist(scoreboard_deaths, 'Death', interval='Weeks')
    plt.show()
d_8 = VBox([out_8], layout=centered)
display(d_8)
html = """
<div class='caption'>
<b>Scores over Time (Deaths)</b>.
(TOP) Scatter plot for all scores as a function of the forecast horizon
(Weekly incidental death forecast scores).
(BOTTOM) Histogram of weekly incidental death forecasts.
</div>
"""
HTML(html)